<a href="https://colab.research.google.com/github/AlexKressner/KI_Logistik_Python/blob/main/G1_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Übersicht
1. [Problem/ Business Understanding](#problem)
1. [Data Understanding & Data Exploration](#data)
  1. [Erster Überblick](#overview)
  1. [Target](#target)
  1. [Features](#feature)
1. [Machine Learning für Regressionsprobleme](#regression)
1. [Machine Learning für Klassifikationsprobleme](#klassifikation)

# Übungsaufgaben
- [Data Exploration - Track & Trace](#tt_eda)
- [Machine Learning - Track & Trace](#tt_ml)
- [Abschlussübung ML](#final)

## 1 Problem/ Business Understanding <a class="anchor" id="problem"></a>
Sie kennen bereits die Daten zu den Fussballweltmeisterschaften von 1930 bis 2014. In diesen ist jedes einzelne Weltmeisterschaftsspiel dokumentiert - von der Zuschauerzahl bis zum Endergebnis. 

Wir werden nun versuchen, die Anzahl der in einem Spiel geschossenen Tore nach Abschluss der ersten Halbzeit vorherzusagen. Durch die Nutzung von maschinellen Lernverfahren versuchen wir aus den zur Halbzeit vorliegenden Daten Muster zu erkennen, die eine zuverlässige Vorhersage der geschossenen Tore erlaubt. 

Zunächst überlegen wir uns, welche Daten (Features) einen Einfluss auf die Anzahl der geschossenen Tore haben. Mit Sicherheit ist in diesem Zusammenhang die Anzahl der zur Halbzeit geschossenen Tore eine wichtige Information. Zusätzlich kann man sich aber auch die folgenden Fragen stellen:
- Hat die Spielpaarung einen Einfluss auf die Anzahl der Tore?
- Hat die Anzahl der Zuschauer einen Einfluss auf die Anzahl der geschossenen Tore?
- Fallen mehr Tore in einem Vorrundenspiel im Vergleich zu einem Halbfinale?
- ...

Sicherlich finden wir auf jede einzelne Frage auch ohne maschinelle Lernverfahren durch eine tiefergehende Datenanalse erste Antworten. Die Bewertung der Einflüsse in ihrer Kombination ist aber schon äußerst schwierig.

Maschinelle Lernverfahren helfen hier enorm. Wir müssen lediglich die Daten in geeigneter Weise aufbereiten, einen passenden Algorithmus auf diese anwenden und erhalten anschließend Prognosen zur interessierten Zielgröße oder Aussagen über die Bedeutung einzelner Features.

**Frage:** Welchem Teilgebiet des maschinellen Lernens ist die oben beschriebene Problemstellung zuzuordnen? Supervised oder Unsupervised Learning?

**Frage:** Handelt es sich um ein Regressions- oder Klassifikationsproblem?

## 2 Data Understanding & Exploration <a class="anchor" id="data"></a>
In einem ersten Schritt ist es wichtig, dass Sie sich einen Überblick zu den vorhandenen Daten verschaffen, d.h.:
- Wie groß ist der Datensatz (Zeilen/Spalten)
- Was bedeuten die einzelnen Werte in den Spalten (Features)?
- Wie sind die Werte der einzelnen Features verteilt?
- Gibt es fehlende Werte, die Sie bereinigen müssen?
- Wie sin die Zusammenhänge zwischen den Features?
- ...

Besonders hilfreich sind an dieser Stelle Visualisierung, die Ihnen einen Überblick zu den Daten geben und bereits etwaige Zusammenhänge zwischen Features und/oder Target aufzeigen!

In [ ]:
import pandas as pd # zur Datenanalyse
import matplotlib.pyplot as plt # zur Datenvisualisierug
import numpy as np

In [ ]:
# Google-Drive einbinden
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Daten laden
data = pd.read_csv("/content/drive/MyDrive/WorldCupMatches.txt")

# Die folgende Datenaufbereitung ist für Sie nicht relevant!
data = data.astype({"Datetime": "M", "RoundID": "O", "MatchID": "O"}, errors='raise') 
data.dropna(inplace=True)
data.replace(' ', np.nan, inplace=True)
data["Hour"] = data["Datetime"].dt.hour
data["Weekday"] = data["Datetime"].dt.day_name()
data.Stage.replace(r'Group(.*)', 'Group', regex=True, inplace=True)
data.Stage.replace('First round', 'Group', inplace=True)
data.Stage.replace('Preliminary round', 'Group', inplace=True)
data.Stage.replace(r'.+third.+', 'Third place', regex=True, inplace=True)
data.head()

### 2.1 Target <a class="anchor" id="target"></a>
Bei dem Target handelt es sich um die Zielgröße, welche wir prognostizieren wollen. In diesem Fall die Anzahl der geschossenen Tore in der regulären Spielzeit plus etwaiger Verlängerung!

In [ ]:
# Berechnung neue Spalte
data["Total Goals"] = data["Home Team Goals"] + data["Away Team Goals"]

In [ ]:
data["Total Goals"].plot(
    kind="hist", 
    title= "Histogramm Total Goals",
    figsize=(8,4),
    color="c",
    edgecolor='k',
    density=True,
    alpha=0.8
    )
plt.plot()

**Frage:** Warum ist es unabdingbar die folgenden Spalten zu löschen?

In [ ]:
data.drop(columns=["Home Team Goals", "Away Team Goals"], inplace=True)

### 2.2 Features <a class="feature" id="overview"></a>
Bei den Features handelt es sich um die Variablen, die zur Vorhersage des Targets verwendet werden - in diesem Fall die Anzahl der geschossenen Tore. 

Zum einen müssen Sie zunächst überlegen, welche der ursprünglichen Features einen Beitrag zur Erklärung des Target leisten können und ob diese noch anzupassen sind. 

Zum anderen leitet man regelmäßig aus den ursprünglichen Features weitere Features ab - in der Annahme, dass diese einen Beitrag zur Vorhersage des Target leisten.

**Relevante Features**

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
irrelevant = ["Year", "City", "Win conditions", 'Referee', 'Stadium',
              'Assistant 1', 'Assistant 2', 'RoundID', 'MatchID',
              "Home Team Initials", "Away Team Initials", "Datetime"
              ]

In [ ]:
# Durch inplace=True müssen Sie nicht schreiben: data = data.drop(...)
data.drop(columns=irrelevant, inplace=True)
data.head()

**Features zu den zur Halbzeit geschossenen Toren**

In [ ]:
data["Half-time Goals"] = data["Half-time Home Goals"] + data["Half-time Away Goals"]
data["Half-time Goals Difference"] = abs(data["Half-time Home Goals"] - data["Half-time Away Goals"])

In [ ]:
data["Half-time Goals Difference"].unique()

In [ ]:
data["Half-time Goals Difference"].value_counts(normalize=True)

Nach den vorausgegangenen Datenmanipulationen erhalten wir nun den folgenden Dataframe (Target + Features), mit dem wir bei der Anwendung maschineller Lernverfahren weiterarbeiten werden.

In [ ]:
data.head()

### 2.3 Korrelationen <a class="feature" id="corr"></a>
Sie erhalten einen ersten sehr guten Eindruck von Zusammenhängen zwischen Features und/ oder Target, wenn Sie Korrelationskoeffizienten berechnen. Mithilfe von `pandas` gelingt dies mühelos.


In [ ]:
correlation_matrix = data.corr()

**Korrelationskoeffizient**

Mit der `pandas`-Funktion `.corr()` berechnen Sie den [Korrelationskoeffizienten nach Pearson](https://de.wikipedia.org/wiki/Korrelationskoeffizient). Dieser misst den "linearen" Zusammenhang zwischen zwei stetigen Merkmalen. Der Koeffizient reicht von 1 bis -1. Für den Fall, dass er nahe 1 liegt, bedeutet dies, dass eine starke positive Korrelation besteht, d.h. "wenn Werte des Merkmals `x` steigen, steigen auch solche des Merkmals `y`"! Der umgekehrte Fall gilt für Koeffizienten, die nahe -1 liegen. Bitte behalten Sie in Erinnerung, dass der Korrelationskoeffizient nach Pearson nur lineare Zusammenhänge erfasst. Regelmäßig finden Sie aber auch nicht-lineare Korrelationen, die dann nicht erfasst werden ([Beispiel](https://de.wikipedia.org/wiki/Korrelationskoeffizient#/media/Datei:Correlation_examples.png)).

In [ ]:
correlation_matrix["Total Goals"].sort_values(ascending=False)

### Übung Datenexploration mit Track & Trace Daten <a class="anchor" id="tt_eda"></a>
**Einführung**

Ein Automobilhersteller hat weltweit verteilte Produktionsstandorte, u.a. auch ein Werk in den USA. Dieses Werk wird mit diversen Bauteilen zur Herstellung von Fahrzeugen aus Europa beliefert. 

Zur Erhöhung der Transparenz in der Supply Chain wurde vor kurzer Zeit Track und Trace System aufgebaut, welches Daten für einzelne Sendungen an definierten Standorten in der Lieferkette aufnimmt. Die Daten finden Sie nachfolgend:

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/KI_LOG_mit_PYTHON/track_trace.txt")
df.head(10)

Jeder einzelne Container (`ContainerNumber`) ist einer Sendung zugeordnet, wobei jede Sendung eine eindeutige Kennzeichnung (`Shipment Number`) aufweist. Zu jeder Sendung werden definierte Events aufgenommen. Diese beinhalten den Namen des Events (`EventName`), die Zeit des Events (`EventTim`) sowie die Lokation des Events (`EventLocation`). Weiterhin ist jeder Sendung ein Schiff (`VesselName`) zugeordnet. In der letzten Spalten finden Sie die Ankunftszeit in **Tagen** (`Time2Arrival`). Diese stellt die Zeit zwischen dem jeweiligen Event und dem Zielevent dar. Der Zielevent ist die Vereinnahmung der Ware (`EventName = Goods Receipt Dock`) im Werk in den USA. Der Datensatz wird um weitere abgeleitete Informationen ergänzt (`EventNameCombined`, `EventTimeMonth`, `EventTimeDay`).


**Aufgabenstellung**

Die Supply Chain Abteilung des Automobilherstellers verfolgt das Ziel, die Daten aus dem Track und Trace System für die Steuerung von Transporten und Beständen zu nutzen. Da es sich um ein neues Projekt zur intelligenten Datenanalyse handelt, wurden Sie gebeten in einem ersten Schritt die zur Verfügung gestellten Daten in Form einer explorativen Datenanalyse aufzubereiten und zu analysieren.

**Konkrete Fragestellungen/ Aufgaben**


1. Wie viele Events finden sich in dem vorliegenden Datensatz?
1. Von welchen deutschen Häfen wird das Werk in Tuscaloosa beliefert? Filtern Sie dazu zunächst auf das Event mit dem Namen `Loading Ship`. Anschließend betrachten Sie die Werte in der Spalte `EventLocation`.
1. Wie ist die mittlere Transportzeit (`Time2Arrival`) von jedem dieser deutschen Häfen zum Werk in den USA? Filtern Sie dazu die `EventLocation` zunächst auf die deutschen Häfen (am besten mithilfe von `isin()`) und speichern Sie die Daten in einer neuen Variablen. Anschließend müssen Sie die Daten nach Häfen gruppieren (`groupby()`) und den Mittelwert der `Time2Arrival` bilden (`mean()`).
1. Erstellen Sie anschließend jeweils ein Histogramm zur `Time2Arrival` für jeden deutschen Hafen. Welcher Hafen hat aus Ihrer Sicht eine bessere Performance?

## 3 Machine Learning für Regressionprobleme <a class="anchor" id="regression"></a>
Wir betrachten nun wieder den Fifa World Cup Datensatz und werden nun wie eingangs beschrieben ein ML-Verfahren verwenden, um die Anzahl der am Ende der regulären Spielzeit (ggf. plus Verlängerung) geschossenen Tore vorherzusagen. Wir nutzen zu diesem Zweck alle Informationen, die uns zur Halbzeit eines Spiels vorliegen.

### 3.1 Datensatz in Features und Target teilen
Zunächst einmal müssen wir unterscheiden, was vorhergesagt werden soll (**Target**) und welche Merkmale dazu genutzt werden sollen (**Features**).

In [ ]:
# Features, d.h. womit lässt sich die Anzahl der Tore prognostizeren!
X = data.drop(columns="Total Goals")
X.head()

In [ ]:
# Target, d.h. was soll prognostiziert werden!
y = data["Total Goals"]
y.head()

In [ ]:
X.shape, y.shape

### 3.2 Datensatz in Trainings- und Testmenge aufteilen

Bevor wir gleich zu der Aufteilung in Trainings- & Testdaten zum Training des ML-Verfahrens kommen, müssen die nicht-numerischen Daten passend aufbereitet werden. Das Vorgehen ist dabei immer gleich. Zur Illustration nutzen wir die Fifa World Cup Daten und dabei die ersten 5 Eintragungen in der Spalte `Home Team Name`.

In [ ]:
example = data["Home Team Name"].head()
example

Wie Ihnen bekannt, enthält die Spalte die jeweilige Heimmannschaft eines Spiels. Wenn wir mit dem Package `sklearn` und den darin verfügbaren ML-Algorithmen arbeiten, müssen wir die Datenstrutkur umwandeln. Dazu nutzen wir die Funktion `get_dummies` wie in der unteren Codezeile dargestellt.

In [ ]:
pd.get_dummies(example)

Wir können die Funktion `get_dummies` auf den gesamten Datensatz anwenden und Pandas sucht sich selbst die umzuwandelnden Spalten.



In [ ]:
X = pd.get_dummies(X)
X.head()

Bei der Aufteilung der Daten in Test- und Trainingsmenge hilft die Funktion `train_test_split`.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Training Algorithmus: X_train (Features), y_train (Target)
# Test des Algorithmus (Bewertung): X_test, y_test
#     1) Algorithmus bekommt Daten (X_test), die er noch nicht kennt & macht Prognose
#     2) Vergleich Prognose mit tatsächlichen Werte (y_test) --> Bewertung
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

**Trainingsdaten:** Das jeweilige ML-Verfahren kennt `y_train` (Target, d.h. Anzahl geschossener Tore) und `X_train` (Features, d.h. die zugehörigen Merkmale des jeweiligen Spiels wie Tore zur Halbzeit etc., die das Target erklären) und versucht einen Zusammenhang zwischen diesen zu lernen!

In [ ]:
y_train.head()

In [ ]:
X_train.head()

### 3.3 Modell auswählen

Für Regressions- und Klassifikationsprobleme gibt es eine Unmenge an ML-Verfahren. Eine Übersicht finden Sie [hier](https://scikit-learn.org/stable/). Wir werden das Verfahren `Random Forest` verwenden. Es gehört zu den besten und robustestens Verfahren im Bereich Machine Learning und wird aus diesem Grund häufig in der Praxis eingesetzt.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = RandomForestRegressor()

### 3.4 Modell trainieren

Das Random Forest Verfahren versucht nun einen Zusammenhang zwischen den Features und dem Target zu lernen und minimiert dabei ein Fehlermaß. In der Grundeinstellung wird die Summe der quadrierten Fehler minimiert, d.h. $∑(y_i - \hat{y}_i)^2$ mit $y=$Anzahl geschossene Tore Spiel $i$ und $\hat{y}=$Prognose Anzahl geschossene Tore Spiel $i$.

In [ ]:
model.fit(X_train, y_train)

### 3.5 Modell anwenden

Zu diesem Zweck nutzen wir nun die **Testdaten**. Das ML-Verfahren erhält die Daten `X_test`, d.h. die Merkmale eines Spiels und wendet die gelernten Zusammenhänge zur Prognose des Targets an, d.h. die Anzahl der geschossenen Tore!

In [ ]:
X_test.head()

In [ ]:
y_pred = model.predict(X_test)
y_pred

### 3.6 Modell evaluieren

Nachdem nun Prognosewerte durch das angelernte Verfahren berechnet wurden, können wir diese mit den tatsächlichen Werten des Target, also der tatsächlichen Anzahl der geschossenen Tore vergleichen (`y_test`). Daraus können wir dann die Güte eines Verfahrens ableiten!

In [ ]:
import sklearn.metrics

In [ ]:
# Genauigkeit des Modells auf der Testmenge 
sklearn.metrics.mean_absolute_error(y_test, y_pred)

In [ ]:
(y_test - y_pred).plot(
    kind="hist",
    title= "Histogramm Prognosefehler",
    figsize=(8,4),
    color="c",
    edgecolor='k',
    density=True,
    alpha=0.8    
    )
plt.show()

### Übung Machine Learning Modell mit Track & Trace Daten <a class="anchor" id="tt_ml"></a>

Sie haben nun erfolgreich eine erste Analyse der Track & Trace Daten gemacht. Sie sollen nun versuchen, die Transportzeit von den europäischen Häfen zum Werk in den USA mithilfe von ML-Verfahren zu prognostizieren. Sollte dies gut gelingen, ist das die Grundlage um die Versorgungssicherheit zu erhöhen und Bestände im Werk zu reduzieren.

Nachfolgend finden Sie die relevanten Daten mit entsprechender Filterung. Gehen Sie nun die Schritte `1-6` aus dem Kapitel "Training eines ML-Verfahrens" durch und wenden Sie es analog auf die Track & Trace Daten an.

In [ ]:
df = df[df["EventName"]=="Loading Ship"]
df.head()

In [ ]:
irrelevant = ["ContainerNumber","ShipmentNumber", "EventTime", "EventCombined"]
df.drop(columns=irrelevant, inplace=True)

In [ ]:
df.head()

**1) Daten in Features und Target aufteilen**

**2) Datensatz in Trainings- und Testmenge aufteilen**

**3) Modell auswählen**

**4) Modell trainieren**

**5) Modell anwenden**

**6) Modell evaluieren**

**Frage:** Welche weiteren Features könnten helfen, um die Prognose der Ankunftszeit zu verbessern?

## 4 Machine Learning für Klassifikationsprobleme <a class="anchor" id="klassifikation"></a>
Im nachfolgenden Beispiel werden wir ein ML-Verfahren einsetzen, um anhand von Untersuchungsergebnissen vorherzusagen, ob ein Patient Diabetes hat. Es handelt sich dabei um eine Klassifikationsproblem (Diabetes/ keine Diabetes). Wir verwenden einen Datensatz mit folgenden Informationen:
1. Anzahl Schwangerschaften
1. Glukosekonzentration nach Glukosetoleranztest
1. Blutdruck (mm Hg)
1. Dicke der Trizepshautfalte (mm)
1. Insulinwert (mu U/ml)
1. Body mass index 
1. Diabetesvorbelastungsfunktion
1. Alter (Jahre)
1. Diabetes (0/1)

Die relevanten Daten haben die folgende Form:

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/KI_LOG_mit_PYTHON/diabetes_clean.txt")
data.head()

### 4.1 Datensatz in Features und Target teilen

Wie bereits zuvor müssen wir unterscheiden, was vorhergesagt werden soll (**Target**) und welche Merkmale dazu genutzt werden sollen (**Features**).

In [ ]:
# Features, d.h. womit lässt sich Diabetes prognostizeren!
X = data.drop(columns="diabetes")
X.head()

In [ ]:
# Target, d.h. was soll prognostiziert werden!
y = data["diabetes"]
y.head()

### 4.2 Datensatz in Trainings- und Testmenge aufteilen
Erneut müssen wir die Daten aufteilen. Da wir ausschließlich mit numerischen Daten arbeiten, müssen wir keine Datenaufbereitung über `pd.get_dummies` vornehmen.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# X und y werden in Trainings- und Testmenge aufgeteilt, so dass die Testmenge 20% der vorhandenen Daten hat
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

In [ ]:
y_train.head()

In [ ]:
X_train.head()

### 4.3 Modell auswählen

Erneut verwenden wir das Verfahren `Random Forest`. Sie müssen allerdings darauf achten, dass Sie einen Random Forest zur Klassifikation importieren. Aus diesem Grund verwenden wir das Verfahren `RandomForestClassifier`. Für die Regressionaufgabe hatten wir das Verfahren `RandomForestRegressor` genutzt. Achten Sie auf diesen wichtigen Unterschied!

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier()

### 4.4 Modell trainieren

Das Random Forest Verfahren versucht nun einen Zusammenhang zwischen den Features und dem Target zu lernen und minimiert dabei ein Fehlermaß. Letzteres misst den Grad der Fehlklassifikation. Beispiel für Fehlklassifikation: Das Modell prognostiziert Diabetes für einen Patienten, in der Realität liegt die Erkrankung aber nicht vor.

In [ ]:
model.fit(X_train, y_train)

### 4.5 Modell anwenden

Das ML-Verfahren erhält die Daten `X_test`, d.h. die Untersuchungsergebnisse mit den relevanten Merkmalen und wendet die gelernten Zusammenhänge zur Prognose des Targets an, d.h. dem Vorhandensein von Diabetes (0/1)!

In [ ]:
X_test.head(2)

In [ ]:
y_pred = model.predict(X_test)

### 4.6 Modell evaluieren

Die Modellbewertung erfolgt bei der Klassifikation regelmäßg nach dem Kriterium der "Accuracy" (andere kommen aber auch zum Einsatz). Die Accuracy setzt die Anzahl der korrekten Klassifikation in das Verhältnis zur Anzahl aller Beobachtungen. Dementsprechend stellt sie den Anteil der korrekten Klassifikationen dar.

In [ ]:
sklearn.metrics.accuracy_score(y_test, y_pred) * 100

Zur Veranschaulichung von Klassifikationsergebnissen hilft regelmäßig eine Konfusionsmatrix. Diese wird wie folgt erstellt:

In [ ]:
conf_matrix = sklearn.metrics.confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = sklearn.metrics.ConfusionMatrixDisplay(conf_matrix, display_labels=model.classes_)
disp.plot()
plt.show()

### 4.7 Beispiel Vorhersage der Anzahl von Schwangerschaften

In [ ]:
# Modelltraining
# 1. Datenaufteilung in X und y
X = data.drop(columns="schwangerschaften")
y = data["schwangerschaften"]
# 2. Datenaufteilung in X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)
# 3. Modell trainieren
model = RandomForestClassifier()
model.fit(X_train, y_train)

In [ ]:
# Modellanwendung Person1
person = data.head(1).drop(columns="schwangerschaften").copy()
person

In [ ]:
model.predict(person)

In [ ]:
# Modellanwendung Person2
person["bmi"]=20
person["diabetes"]=0
person["alter"]=25

In [ ]:
model.predict(person)

### Abschlussübung Machine Learning <a class="anchor" id="final"></a>
Wir bleiben beim Datensatz zur Diabetes. Entwickeln Sie ein Machine Learning Modell, welches den BMI einer untersuchten Person auf Basis der Untersuchungsergebnisse (inkl. Diabetesbefund) vorhersagt!